<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="HEIG-VD Logo" width="100"/>

# Cours TAL - Laboratoire 6
# Trois méthodes de désambiguïsation lexicale

**Objectif**

L'objectif de ce laboratoire est d'implémenter et de comparer plusieurs méthodes de désambiguïsation lexicale (en anglais, *Word Sense Disambiguation* ou WSD).  Vous utiliserez un corpus avec plusieurs milliers de phrases, chaque phrase contenant une occurrence du mot anglais *interest* annotée avec le sens que ce mot possède dans la phrase respective.  Les trois méthodes sont les suivantes (elles seront détaillées par la suite) :

* Algorithme de Lesk simplifié.
* Utilisation de word2vec.
* Classification supervisée utilisant des traits lexicaux :
   - les mots en position -1, -2, ..., et +1, +2, ..., par rapport à *interest* ;
   - apparition de mots indicateurs dans le voisinage de *interest*.

Les deux premières méthodes n'utilisent pas l'apprentissage automatique.  Elles fonctionnent selon le même principe : comparer le contexte d'une occurrence de *interest* avec chacune des définitions des sens (*synsets*) et choisir la définition la plus proche du contexte.  L'algorithme de Lesk définit la proximité comme le nombre de mots en commun, alors que word2vec la calcule comme la similarité de vecteurs.  La dernière méthode vise à classifier les occurrences de *interest*, les sens étant les classes, et les attributs étant les mots du contexte (apprentissage supervisé).

## 1. Analyse des données

Téléchargez le corpus *interest* depuis le [site du Prof. Ted Pedersen](http://www.d.umn.edu/~tpederse/data.html) (il se trouve en bas de la page).  Téléchargez l'archive ZIP marquée *original format without POS tags* et extrayez le fichier `interest-original.txt`.  Téléchargez également le fichier `README.int.txt` indiqué à la ligne au-dessus. Veuillez répondre brièvement aux questions suivantes :

a. Quelles sont les URL du fichier ZIP et celle du fichier `README.int.txt` ?

b. Quel est le format du fichier `interest-original.txt` et comment sont annotés les sens de *interest* ?

c. Est-ce qu'on considère aussi les occurrences au pluriel (*interests*) ?

d. Comment sont annotées les phrases qui contiennent plusieurs occurrences du mot *interest* ?

In [1]:
# Veuillez répondre ici (en commentaire) aux questions.
#a. ZIP = https://www.d.umn.edu/~tpederse/Data/interest-original.nopos.tar.gz
#   txt = https://www.d.umn.edu/~tpederse/Data/README.int.txt

#b. il s'agit d'un fichier au format .txt. Chaque occurrence du mot "interest" est suffixée avec un tiret bas suivit d'un chiffre correspondant à un sens particulier du mot.

#c. Oui, on considère aussi les occurrences au pluriel.

#d. Ces phrases sont généralement dédoublées. On annote l'occurrence du mot qu'on souhaite traitée comme précédemment et on préfixe les autres occurrence avec un astérisque. On dédouble la phrase pour autant de mots "interest" qu'elle contient.
# Exemple:
# in  such circumstances  ,  free-speech interests_4  outweighed  the *interests  of  the copyright owner  . 
# in  such circumstances  ,  free-speech *interests  outweighed  the interests_4  of  the copyright owner  . 

**1e.** D'après le fichier `README.int.txt`, quelles sont les définitions des six sens de *interest* annotés dans les données et quelles sont leurs fréquences ? Vous pouvez copier/coller l'extrait de `README`ici.

In [2]:
# Sense 1 = readiness to give attention
# Sense 2 = quality of causing attention to be given to
# Sense 3 = activity, etc. that one gives attention to
# Sense 4 = advantage, advancement or favor
# Sense 5 = a share in a company or business
# Sense 6 = money paid for the use of money

**1f.** De quel dictionnaire viennent les sens précédents ? Où peut-on le consulter en ligne ?  Veuillez aligner les définitions du dictionnaire avec les six sens annotés en écrivant par exemple `Sense 3 = "an activity that you enjoy doing or a subject that you enjoy studying"`.

In [3]:
# Ils viennent de la version électronique de la première édition du Longman's Dictionary of Contemporary English (LDOCE)
# Consultable en ligne ici: https://www.ldoceonline.com/
# Sense 1 = "if you have an interest in something or someone, you want to know or learn more about them"
# Sense 2 = "a quality or feature of something that attracts your attention or makes you want to know more about it"
# Sense 3 = "an activity that you enjoy doing or a subject that you enjoy studying"
# Sense 4 = "the things that bring advantages to someone or something"
# Sense 5 = "if you have an interest in a particular company or industry, you own shares in it"
# Sense 6 = "the extra money that you must pay back when you borrow money"

**1g.** En consultant [WordNet en ligne](http://wordnetweb.princeton.edu/perl/webwn), trouvez les définitions des synsets  pour le **nom commun** *interest*.  Combien de synsets y a-t-il ?  Veuillez indiquer comme avant la **définition** de chaque synset pour chacun des six sens ci-dessus (au besoin, fusionner ou ignorer des synsets).

In [4]:
# Il y a 7 synsets pour le nom commun "interest" dans WordNet.
# Sense 1 = "a sense of concern with and curiosity about someone or something"
# Sense 2 = "the power of attracting or holding one's attention (because it is unusual or exciting etc.)"
# Sense 3 = "a diversion that occupies one's time and thoughts (usually pleasantly)"
# Sense 4 = "a reason for wanting something done"
# Sense 5 = "(law) a right or legal share of something; a financial involvement with something"
# Sense 6 = "a fixed charge for borrowing money; usually a percentage of the amount borrowed"

**1h.** Définissez (manuellement, ou avec quelques lignes de code) une liste nommée `senses1` avec les mots des définitions du README, en supprimant les stopwords (p.ex. les mots < 4 lettres).  Affichez la liste.

In [5]:
import nltk
from nltk import NaiveBayesClassifier

In [6]:
from random import randrange

In [7]:
# Veuillez répondre ici à la question et créer la variable 'senses1' (liste de 6 listes de chaînes).
senses1 = [
    ["readiness", "give", "attention"],
    ["quality", "causing", "attention", "given"],
    ["activity", "that", "gives", "attention"],
    ["advantage", "advancement", "favor"],
    ["share", "company", "business"],
    ["money", "paid", "money"]
]

print(senses1)

[['readiness', 'give', 'attention'], ['quality', 'causing', 'attention', 'given'], ['activity', 'that', 'gives', 'attention'], ['advantage', 'advancement', 'favor'], ['share', 'company', 'business'], ['money', 'paid', 'money']]


**1i.** En combinant les définitions obtenues aux points (4) et (5) ci-dessus, construisez une liste nommée `senses2` avec pour chacun des sens de *interest* une liste de **mots-clés** correspondants.  Vous pouvez concaténer les définitions, puis écrire des instructions en Python pour extraire les mots (uniques).  Respectez l'ordre des sens données par `README`, et à la fin affichez `senses2`.

In [8]:
# Veuillez répondre ici à la question et créer la variable 'senses2' (liste de 6 listes de chaînes).
senses_ldce = [
    "if you have an interest in something or someone, you want to know or learn more about them",
    "a quality or feature of x§something that attracts your attention or makes you want to know more about it",
    "an activity that you enjoy doing or a subject that you enjoy studying",
    "the things that bring advantages to someone or something",
    "if you have an interest in a particular company or industry, you own shares in it",
    "the extra money that you must pay back when you borrow money"
]

senses_wordnet = [
    "a sense of concern with and curiosity about someone or something",
    "the power of attracting or holding one's attention (because it is unusual or exciting etc.)",
    "a diversion that occupies one's time and thoughts (usually pleasantly)",
    "a reason for wanting something done",
    "(law) a right or legal share of something; a financial involvement with something",
    "a fixed charge for borrowing money; usually a percentage of the amount borrowed"
]

senses2 = []
for i in range(min(len(senses_ldce), len(senses_wordnet))):
    words = nltk.word_tokenize(senses_ldce[i] + " " + senses_wordnet[i])
    words = [word for word in words if len(word) > 4]
    words = list(set(words))
    senses2.append(words)

print(senses2)

[['about', 'sense', 'curiosity', 'learn', 'someone', 'concern', 'interest', 'something'], ['x§something', 'quality', 'about', 'because', 'feature', 'attention', 'attracts', 'holding', 'unusual', 'exciting', 'makes', 'attracting', 'power'], ['pleasantly', 'occupies', 'doing', 'studying', 'thoughts', 'usually', 'enjoy', 'activity', 'subject', 'diversion'], ['bring', 'wanting', 'reason', 'someone', 'things', 'something', 'advantages'], ['shares', 'company', 'right', 'industry', 'legal', 'share', 'involvement', 'particular', 'financial', 'interest', 'something'], ['charge', 'amount', 'borrowed', 'borrow', 'percentage', 'usually', 'money', 'fixed', 'borrowing', 'extra']]


**1j.** Chargez les données depuis `interest-original.txt` dans une liste appelée `sentences` qui contient pour chaque phrase la liste des mots (sans les séparateurs *$$* et *===...*).  Ces phrases sont-elles déjà tokenisées en mots ?  Sinon, faites-le.  À ce stade, ne modifiez pas encore les occurrences annotées *interest(s)\_X*.  Comptez le nombre total de phrases et affichez-en trois au hasard.

In [9]:
# Veuillez répondre ici à la question.
from nltk.tokenize import RegexpTokenizer 

#Nous avons retiré la ponctuation et les guillemets qui ne sont pas pertinents pour la désambiguïsation
tokenizer = RegexpTokenizer(r"\w+(?:'\w+)?|'\w+")
sentences: list[list[str]] = []
with open("./interest-corpus/interest-original.txt", "r") as f:
    for line in f:
        if not line.startswith("$$"):
            words = tokenizer.tokenize(line)
            sentences.append(words)
        

print("Il y a {} phrases.\nEn voici 3 au hasard :".format(len(sentences)))
print(sentences[151:154])
# print(sentences[73])

Il y a 2368 phrases.
En voici 3 au hasard :
[['investor', 'interest_1', 'in', 'stock', 'funds', 'has', "n't", 'stalled', 'at', 'all', 'mr', 'hines', 'maintains'], ['it', 'is', 'in', 'the', 'western', 'interest_4', 'to', 'see', 'mr', 'gorbachev', 'succeed'], ['revco', 'insists', 'that', 'the', 'proposal', 'is', 'simply', 'an', 'expression', 'of', 'interest_1', 'because', 'under', 'chapter', '11', 'revco', 'has', 'exclusivity', 'rights', 'until', 'feb', '28']]


## 2. Algorithme de Lesk simplifié

**2a.** Définissez une fonction `wsd_lesk(senses, sentence)` qui prend deux arguments : une liste de listes de mots-clés (comme `senses1` et `senses2` ci-dessus) et une phrase avec une occurrence annotée de *interest* ou *interests*, et qui retourne l'index du sens le plus probable (entre 1 et 6) selon l'algorithme de Lesk.  Cet algorithme choisit le sens qui a le maximum de mots en commun avec le contexte de *interest*.  Vous pouvez choisir vous-mêmes la taille de ce voisinage (`window_size`).  En cas d'égalité entre deux sens, tirer la réponse au sort.

In [10]:
# Veuillez répondre ici à la question.
WINDOW_SIZE = 8

def get_neighboring_words(tokenized_sentence, window_size):
    target_word_index = None
    for i, word in enumerate(tokenized_sentence):
        if word.startswith('interest_') or word.startswith('interests_'):
            target_word_index = i
            break
    
    if target_word_index is None:
        return []  # Si aucun mot ne correspond au critère
    
    start_index = max(0, target_word_index - window_size)
    end_index = min(len(tokenized_sentence), target_word_index + window_size + 1)
    return tokenized_sentence[start_index:target_word_index] + tokenized_sentence[target_word_index+1:end_index]

def wsd_lesk(senses, sentence):
    best_sense = -1
    best_overlap = 0
    sentence = get_neighboring_words(sentence, WINDOW_SIZE)
    
    for i, sense in enumerate(senses):
        overlap = len(set(sense) & set(sentence))
        if overlap > best_overlap:
            best_overlap = overlap
            best_sense = i
            
    if best_sense == -1:
        #Aucun sens n'a été trouvé. On en sélectionne 1 parmi les 6
        return randrange(6) + 1
            
    best_senses = [i for i, sense in enumerate(senses) if len(set(sense) & set(sentence)) == best_overlap]
    
    if len(best_senses) > 1:
        #Deux ou plusieurs sens ont le même score. On en sélectionne 1 au hasard
        best_sense = best_senses[randrange(len(best_senses))]
        
    return best_sense + 1

**2b.** Définissez maintenant une fonction `evaluate_wsd(fct_name, senses, sentences)` qui prend en paramètre le nom de la méthode de similarité (pour commencer : `wsd_lesk`) ainsi que la liste des mots-clés par sens, et la liste de phrases, et qui retourne le score de la méthode de similarité.  Ce score sera tout simplement le pourcentage de réponses correctes (sens trouvé identique au sens annoté).

In [11]:
# Veuillez répondre ici à la question.
import re

# Extrait le numéro du sens du mot interest.
def extract_sense_number(word_list):
    interest_number = None
    for word in word_list:
        match = re.match(r'interests?_(\d+)', word)
        if match:
            interest_number = int(match.group(1))
            break
    
    return interest_number

def evaluate_wsd(fct_name, senses, sentences):
    correct = 0
    for sentence in sentences:
        sense = extract_sense_number(sentence)
        if fct_name(senses, sentence) == sense:
            correct += 1
    return correct / len(sentences)

print("Taille de la fenêtre: ", WINDOW_SIZE)
print("Score avec sense1: ", evaluate_wsd(wsd_lesk, senses1, sentences))
print("Score avec sense2: ", evaluate_wsd(wsd_lesk, senses2, sentences))

Taille de la fenêtre:  8
Score avec sense1:  0.17567567567567569
Score avec sense2:  0.20819256756756757


**2c.** En fixant au mieux la taille de la fenêtre autour de *interest*, quel est le meilleur score de la méthode de Lesk simplifiée ?  Quelle liste de sens conduit à de meilleurs scores, `senses1` ou `senses2` ?

In [12]:
# Veuillez répondre ici à la question.
# Le meilleur score obtenu est de l'ordre de 0.222
# Le meilleur score est obtenu avec la liste de sense 2, ce qui semble logique étant donné qu'elle est plus fournie. Notons toutefois qu'avec notre implémentation, la taille des fenêtres n'a pas beaucoup d'intérêt à être de petite taille puisqu'elle s'applique par phrase et non pas sur le corpus total. À vrai dire, le problème principal de ce modèle est qu'une grande majorité d'attribution des sens se fait au hasard, ce qui est surtout visible avec sense1 qui a un score de ~1/6. En gardant cette valeur en tête, on peut dire que sense2 est à peine plus performant et que cette méthode n'est pas du tout adaptée à notre contexte.

## 3. Utilisation de word2vec pour la similarité contexte vs. synset

**3a.** En réutilisant une partie du code de `wsd_lesk`, veuillez maintenant définir une fonction `wsd_word2vec(senses, sentence)` qui choisit le sens en utilisant la similarité **word2vec** étudiée dans le labo précédent. 
* Vous pouvez chercher dans la [documentation des KeyedVectors](https://radimrehurek.com/gensim/models/keyedvectors.html) comment calculer directement la similarité entre deux listes de mots.
* Comme `wsd_lesk`, la nouvelle fonction `wsd_word2vec` prend en argument une liste de listes de mots-clés par sens (comme `senses1` et `senses2` ci-dessus), et une phrase avec une occurrence annotée de *interest* ou *interests*.
* La fonction retourne le numéro du sens le plus probable selon la similarité word2vec entre les mots du sens et ceux du voisinage de *interest*.  En cas d'égalité, tirer le sens au sort.
* Vous pouvez régler la taille du voisinage (`window_size`) par l'expérimentation.  

In [13]:
import gensim
path_to_model = "./GoogleNews-vectors-negative300.bin" # à adapter
wv_model = gensim.models.KeyedVectors.load_word2vec_format(path_to_model, binary=True)  # C bin format

In [14]:
# Veuillez répondre ici à la question.
WINDOW_SIZE_W2V = 4

def wsd_word2vec(senses, sentence):
    best_sense = -1
    best_similarity = 0
    sentence = get_neighboring_words(sentence, WINDOW_SIZE_W2V)
    
    for i, sense in enumerate(senses):
        similarity = wv_model.n_similarity(sense, sentence)
        if similarity > best_similarity:
            best_similarity = similarity
            best_sense = i
            
    # Le cas où deux sens ont une similarité égale est quasi inexistant. Mais la consigne l'exige, alors voici la gestions dudit cas:        
    if best_sense == -1:
        #Aucun sens n'a été trouvé. On en sélectionne 1 parmi les 6
        return randrange(6) + 1
            
    best_senses = [i for i, sense in enumerate(senses) if wv_model.n_similarity(sense, sentence) == best_similarity]
    
    if len(best_senses) > 1:
        #Deux ou plusieurs sens ont le même score. On en sélectionne 1 au hasard
        best_sense = best_senses[randrange(len(best_senses))]
        
    return best_sense + 1

**3b.** Appliquez maintenant la même méthode `evaluate_wsd` avec la fonction `wsd_word2vec` (en cherchant une bonne valeur de la taille de la fenêtre) et affichez le score de la similarité word2vec.  Comment se compare-t-il avec le score précédent (Lesk) ?

In [15]:
# Veuillez répondre ici à la question.
print("Taille de la fenêtre: ", WINDOW_SIZE_W2V)
print("Score avec sense1: ", evaluate_wsd(wsd_word2vec, senses1, sentences))
print("Score avec sense2: ", evaluate_wsd(wsd_word2vec, senses2, sentences))

# Contraitement au score de Lesk, la taille de la fenêtre est plus significative et le résultat de sense2 diminue au delà de 4 mots voisin. De plus, avec le calcul de similarité, nous ne rencontrons quasi jamais de cas où deux sens ont une similarité égale, annulant ainsi les résultats corrects obtenus uniquement par hasard. 
# Cette méthode garantit ainsi des résultats plus fiables et plus précis. Nous pouvons mieux statuer sur la réelle performance de ce modèle.

Taille de la fenêtre:  4
Score avec sense1:  0.28969594594594594
Score avec sense2:  0.5502533783783784


## 4. Classification supervisée avec des traits lexicaux
Vous entraînerez maintenant des classifieurs pour prédire le sens d'une occurrence dans une phrase.  Le principal défi sera de transformer chaque phrase en un ensemble de traits (attributs, *features*), pour créer les données en vue des expériences de classification.

Veuillez utiliser le classifieur `NaiveBayesClassifier` fourni par NLTK.  Le mode d'emploi se trouve dans le [Chapitre 6, sections 1.1-1.3](https://www.nltk.org/book/ch06.html) du livre NLTK.  Consultez-le attentivement pour trouver comment formater les données.  De plus, il faudra séparer les données en sous-ensembles d'entraînement et de test.

### 4.1. Attributs lexicaux positionnels

Dans cette première représentation des attributs, vous les coderez comme `mot-2`, `mot-1`, `mot+1`, `mot+2`, etc. (fenêtre de taille `2*window_size` autour de *interest*) et vous leur donnerez les valeurs des mots observés aux emplacements respectifs, ou `NONE` si la fenêtre dépasse l'étendue de la phrase.  Vous ajouterez un attribut qui est le mot *interest* lui-même, qui peut être au singulier ou au pluriel.  Pour chaque occurrence de *interest*, vous devrez donc créer une représentation formelle, incluant un dictionnaire Python et le numéro du sens :
```
[{'word-1': 'in', 'word+1': 'rates', 'word-2': 'declines', 'word+2': 'NONE', 'word0': 'interest'}, 6]
```
Vous regrouperez toutes ces entrées dans une liste totale appelée `items_with_features_A`.  (Le numéro du sens servira à l'entraînement, puis il sera caché à l'évaluation, quand on comparera la prédiction du système au numéro correct.)  

**4.1a.** En partant de la liste des phrases appelée `sentences`(préparée plus haut), veuillez générer ici cette liste totale, en vous aidant si nécessaire du livre NLTK.

In [79]:
from typing import Iterable
WINDOW_SIZE_NLTK = 2
items_with_features_A =  []

for sentence in sentences:
    sentence_feature = [{}, 0]
    length = len(sentence)
    pos_needle = 0
    # find the word's position in each sentence
    for i in range(length):
        if sentence[i].startswith("interest"):
            pos_needle = i
            break
        
    # there are 4 words in the txt that are "Interest" but are not tagged. This seems like an error so we ignore them
    if not sentence[pos_needle][-1].isdigit():
        continue
    sense_kind = int(sentence[pos_needle][-1])
    sentence_feature[1] = sense_kind
    
    for pos in range(-WINDOW_SIZE_NLTK, WINDOW_SIZE_NLTK + 1):
        word_pos = pos_needle + pos
        sign = "+" if pos > 0 else ""
        sentence_feature[0][f"word{sign}{pos}"] = "NONE" if word_pos < 0 or word_pos >= length else (sentence[word_pos][:-2] if pos == 0 else sentence[word_pos])
        
    items_with_features_A.append(sentence_feature)
        
print(items_with_features_A[1])
print(items_with_features_A[151:154])

[{'word-2': 'indicate', 'word-1': 'declining', 'word0': 'interest', 'word+1': 'rates', 'word+2': 'because'}, 6]
[[{'word-2': 'be', 'word-1': 'of', 'word0': 'interest', 'word+1': 'to', 'word+2': 'cbs'}, 2], [{'word-2': 'NONE', 'word-1': 'but', 'word0': 'interest', 'word+1': 'instead', 'word+2': 'decreased'}, 1], [{'word-2': 'NONE', 'word-1': 'but', 'word0': 'interest', 'word+1': 'waned', 'word+2': 'as'}, 1]]


**4.1b.** On souhaite maintenant entraîner un classifieur sur une partie des données, et le tester sur une autre.  Vous garderez 80% des données pour l'entraînement et utiliserez les 20% restants pour l'évaluation.  Veuillez faire cette division séparément pour chaque sens, pour que les deux ensembles contiennent les mêmes proportions de sens que l'ensemble de départ ("stratification"), et enregistrer les deux sous-ensembles de `items_with_features_A` sous les noms respectifs de `iwf_A_train` et `iwf_A_test`.

In [80]:
from random import shuffle

In [89]:
from random import shuffle
from collections import defaultdict


# Initialisation des listes pour l'entraînement et le test
iwf_A_train = []
iwf_A_test = []

# Création d'un dictionnaire pour stocker les items par sens
items_by_sense = defaultdict(list)

# Regroupement des items par sens
for item in items_with_features_A:
    sense = item[1]
    items_by_sense[sense].append(item)

# Séparation des données en ensembles d'entraînement et de test
for sense, items in items_by_sense.items():
    shuffle(items)
    split_point = int(0.8 * len(items))
    iwf_A_train.extend(items[:split_point])
    iwf_A_test.extend(items[split_point:])

# Affichage des résultats
print(len(iwf_A_train), ' ', len(iwf_A_test))
print(iwf_A_test[:2], iwf_A_test[-2:])

# Vérification des proportions
from collections import Counter

train_senses = [item[1] for item in iwf_A_train]
test_senses = [item[1] for item in iwf_A_test]

print("Distribution dans l'ensemble d'entraînement:", Counter(train_senses))
print("Distribution dans l'ensemble de test:", Counter(test_senses))

1852   465
[[{'word-2': 'to', 'word-1': 'allow', 'word0': 'interest', 'word+1': 'rates', 'word+2': 'to'}, 6], [{'word-2': 'year', 'word-1': 'in', 'word0': 'interest', 'word+1': 'expense', 'word+2': 'NONE'}, 6]] [[{'word-2': 'in', 'word-1': 'national', 'word0': 'interest', 'word+1': 'because', 'word+2': 'both'}, 2], [{'word-2': 'is', 'word-1': 'of', 'word0': 'interest', 'word+1': 'to', 'word+2': 'the'}, 2]]
Distribution dans l'ensemble d'entraînement: Counter({6: 988, 5: 385, 1: 284, 4: 135, 3: 52, 2: 8})
Distribution dans l'ensemble de test: Counter({6: 247, 5: 97, 1: 71, 4: 34, 3: 13, 2: 3})


**4.1c.** Veuillez créer une instance de `NaiveBayesClassifier`, l'entraîner sur `iwf_A_train` et la tester sur `iwf_A_test` (voir la documentation NLTK).  En expérimentant avec différentes largeurs de fenêtres, quel est le meilleur score global que vous obtenez (avec la fonction `accuracy`), et comment se compare-t-il avec les précédents ?

In [83]:
from nltk.classify import naivebayes
from nltk.classify.util import accuracy

classifier = naivebayes.NaiveBayesClassifier.train(iwf_A_train)
test_accuracy = accuracy(classifier, iwf_A_test)
print(f"Accuracy: {test_accuracy:.4f}")
 
# On a une bien meilleure précisiion qu'auparavant. 0.88 est un bon score.
# Avec une taille de 3 mots autour du mot d'intérêt, on a une amélioration de 0.005. C'est parfaitement négligeable.
# Avec une taille de 1, on passe à 0.83.
# Les valeurs supérieures à trois ne semblent pas non plus avoir des améliorations significatives. Une taille de deux semblent être correctes.




Accuracy: 0.8882


**4.1d.** En utilisant la fonction `show_most_informative_features()`, veuillez afficher les attributs les plus informatifs et commenter le résultat.

In [91]:
classifier.show_most_informative_features()

# Ce tableau est de la forme suivante : 
# [feature]  [sense 1 : sense 2] [ratio du sens 1 par rapport au sense 2]
# Donc si le ratio du sense 1 par rapport au sense 2 est grand, il a donc plus de chance que si la feature "word+n" est "x" soit de sense 1 plutôt que de sense 2.

# Ce qu'on peut observer de ce tableau:
# - Les mots les plus importants sont les mots les plus proches.
# - Lorsque "interest" est au pluriel, il y a  bien plus de chance que le sense de ce mot soit le troisième que le premier (et tous les autres après analyse du corpus). 
#   Par contre, au singulier, ce n'est pas le cas.
# - Le mot "in" est plutôt important pour déterminer le sense





Most Informative Features
                  word+1 = 'in'                5 : 6      =     70.9 : 1.0
                   word0 = 'interests'         3 : 1      =     63.8 : 1.0
                  word-1 = 'other'             3 : 6      =     46.6 : 1.0
                  word+1 = 'of'                4 : 6      =     28.6 : 1.0
                  word+2 = 'and'               6 : 5      =     24.1 : 1.0
                  word+2 = 'on'                6 : 5      =     19.8 : 1.0
                  word-1 = 'in'                6 : 5      =     19.1 : 1.0
                  word-2 = 'company'           5 : 6      =     16.8 : 1.0
                  word-2 = 'NONE'              6 : 4      =     15.8 : 1.0
                  word-1 = 'and'               6 : 5      =     15.8 : 1.0


### 4.2 Présence de mots indicateurs

Une deuxième façon d'encoder les attributs lexicaux est de constituer un vocabulaire avec tous les mots qui apparaissent dans tous les voisinages de *interest* et de définir ces mots comme attributs.  Par conséquent, pour chaque occurrence de *interest*, on extrait la valeur de ces attributs sous la forme :
```
[{('rate' : True), ('in' : False), ...}, 1]
```
où *'rate'*, *'in'*, etc., sont les mots du vocabulaire, True/False indiquent leur présence/absence autour de l'occurrence de *interest* qui nous intéresse, et le dernier nombre est le sens, entre 1 et 6.

**4.2a.** Pour commencer, en partant de `sentences` et en fixant la taille de la fenêtre, veuillez constituer la liste de tous les mots observés autour de tous les voisinages de toutes les occurrences de *interest*.

In [131]:
from collections import defaultdict
WINDOW_SIZE_NLTK_2 = 2
word_list = []

for sentence in sentences:
    for i, word in enumerate(sentence):
        if word.startswith("interest"):
            start_index = max(0, i - WINDOW_SIZE_NLTK_2)
            end_index = min(len(sentence), i + WINDOW_SIZE_NLTK_2 + 1)
            context_words = sentence[start_index:i] + sentence[i+1:end_index]
            word_list.extend(context_words)

print(len(word_list))
print(word_list[:50])

9449
['declines', 'in', 'rates', 'indicate', 'declining', 'rates', 'because', 'short', 'term', 'rates', '83', '4', 'in', 'this', 'company', 'with', 'in', 'the', 'refunded', 'plus', 'set', 'the', 'rate', 'on', "'s", 'own', 'prompted', 'the', 'principal', 'and', 'is', 'the', 'increase', 'its', 'to', '70', 'the', 'strong', 'of', 'japanese', 'early', 'if', 'rates', 'decline', 'drop', 'in', 'rates', 'since', 'drop', 'in']


**4.2b.** En utilisant par exemple un objet de type `nltk.FreqDist`, veuillez sélectioner les 500 mots les plus fréquents (vous pourrez aussi optimiser ce nombre), dans une liste appelée `vocabulary`.  À votre avis, est-ce une bonne idée d'enlever les *stopwords* de cette liste pour construire les traits ?

In [145]:
# Veuillez répondre ici à la question.
vocabulary = nltk.FreqDist(word_list).most_common(800)
print(vocabulary[:50])

# Ce n'est pas une bonne idée car les stopword peuvent changer le sense du mot interest.
# Exemple en français mais peut s'appliquer ici aussi : "une équivalence" pourrait désigner la reconnaissance d'un diplôme étranger. Ce sens est possible grâce aux déterminants qui seraient, en temps normal, considérés comme un stop word 


[('in', 722), ('rates', 638), ('the', 459), ('and', 309), ('of', 281), ('to', 231), ('a', 167), ('on', 156), ('rate', 155), ('short', 108), ('its', 106), ('payments', 106), ("'s", 105), ('an', 84), ('lower', 74), ('with', 72), ('has', 68), ('that', 68), ('for', 66), ('by', 58), ('is', 56), ('high', 54), ('are', 53), ('annual', 51), ('term', 50), ('u', 50), ('from', 50), ('will', 47), ('foreign', 45), ('have', 45), ('minority', 44), ('or', 43), ('their', 43), ('income', 43), ('higher', 42), ('at', 40), ('s', 39), ('as', 38), ('below', 37), ('other', 36), ('net', 35), ('buying', 35), ('bonds', 34), ('pay', 33), ('which', 31), ('accrued', 28), ('current', 28), ('investor', 27), ('plus', 26), ('but', 25)]


**4.2c.** Veuillez maintenant créer l'ensemble total de données formatées, en convertissant chaque phrase contenant une occurrence de *interest* à un dictionnaire de traits/valeurs (suivi du numéro du sens), comme exemplifié au début de cette section 4.2.  Cet ensemble sera appelé `items_with_features_B`.

In [146]:
items_with_features_B = []
# [{('rate' : True), ('in' : False), ...}, 1]
# Veuillez répondre ici à la question.
for sentence in sentences:
    sentence_feature = [{}, 0]
    length = len(sentence)
    pos_needle = 0
    # find the word's position in each sentence
    for i in range(length):
        if sentence[i].startswith("interest"):
            pos_needle = i
            break

    # there are 4 words in the txt that are "Interest" but are not tagged. This seems like an error so we ignore them
    if not sentence[pos_needle][-1].isdigit():
        continue
        
    sense_kind = int(sentence[pos_needle][-1])
    sentence_feature[1] = sense_kind

    lower = max(0, pos_needle - WINDOW_SIZE_NLTK)
    upper = min(length - 1, pos_needle + WINDOW_SIZE_NLTK)

    bordering_words = sentence[lower:upper + 1]

    for word, _ in vocabulary:
        sentence_feature[0][word] = word in bordering_words
    
    items_with_features_B.append(sentence_feature)
print(len(items_with_features_B))

2317


**4.2d.** Comme dans la section 4.1, veuillez créer maintenant deux sous-ensembles de `items_with_features_B` appelés `iwf_B_train` (80% des items) et `iwf_B_test` (20% des items), avec une sélection aléatoire mais stratifiée.

In [147]:
from random import shuffle
from collections import defaultdict


# Initialisation des listes pour l'entraînement et le test
iwf_B_train = []
iwf_B_test = []

# Création d'un dictionnaire pour stocker les items par sens
items_by_sense = defaultdict(list)

# Regroupement des items par sens
for item in items_with_features_B:
    sense = item[1]
    items_by_sense[sense].append(item)

# Séparation des données en ensembles d'entraînement et de test
for sense, items in items_by_sense.items():
    shuffle(items)
    split_point = int(0.8 * len(items))
    iwf_B_train.extend(items[:split_point])
    iwf_B_test.extend(items[split_point:])

print(len(iwf_B_train), ' ', len(iwf_B_test))

1852   465


**4.2e.** Comme dans la section 4.1, veuillez créer une instance de `NaiveBayesClassifier`, l'entraîner sur `iwf_B_train` et la tester sur `iwf_B_test`.  En expérimentant avec différentes largeurs de fenêtres et tailles du vocabulaire, quel est le meilleur score que vous obtenez, et comment se compare-t-il avec les précédents ?

In [148]:
classifier = naivebayes.NaiveBayesClassifier.train(iwf_B_train)
test_accuracy = accuracy(classifier, iwf_B_test)
print(f"Accuracy: {test_accuracy:.4f}")


Accuracy: 0.9032


**4.2f.** Quels sont les attributs les plus informatifs ?

In [149]:
classifier.show_most_informative_features(20)

Most Informative Features
                    home = True                2 : 6      =    109.9 : 1.0
                business = True                3 : 6      =     93.3 : 1.0
                    some = True                2 : 6      =     65.9 : 1.0
                 because = True                2 : 5      =     42.9 : 1.0
                    both = True                2 : 5      =     42.9 : 1.0
                 general = True                2 : 5      =     42.9 : 1.0
                   other = True                3 : 1      =     41.9 : 1.0
                      be = True                2 : 1      =     31.7 : 1.0
              especially = True                2 : 1      =     31.7 : 1.0
                national = True                2 : 1      =     31.7 : 1.0
                     can = True                3 : 6      =     31.1 : 1.0
                     our = True                3 : 6      =     31.1 : 1.0
                      on = True                6 : 5      =     27.7 : 1.0

## 5. Conclusion

Veuillez recopier ci-dessous, en guise de conclusion, les scores des quatre expériences réalisées, pour pouvoir les comparer d'un coup d'oeil.  Quel est le meilleur score obtenu?

## Fin du laboratoire

Merci de nettoyer votre feuille, sauvegarder le résultat, et soumettre le *notebook* sur Cyberlearn.